<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1">Preparation</a></span><ul class="toc-item"><li><span><a href="#Load-files" data-toc-modified-id="Load-files-1.1">Load files</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-2.1">Helper functions</a></span></li><li><span><a href="#Metric-functions" data-toc-modified-id="Metric-functions-2.2">Metric functions</a></span><ul class="toc-item"><li><span><a href="#All-metrics" data-toc-modified-id="All-metrics-2.2.1">All metrics</a></span></li><li><span><a href="#Gaze-data" data-toc-modified-id="Gaze-data-2.2.2">Gaze data</a></span></li><li><span><a href="#Fixation-data" data-toc-modified-id="Fixation-data-2.2.3">Fixation data</a></span></li><li><span><a href="#Event-data" data-toc-modified-id="Event-data-2.2.4">Event data</a></span></li></ul></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3">Analysis</a></span><ul class="toc-item"><li><span><a href="#Baseline-Analysis" data-toc-modified-id="Baseline-Analysis-3.1">Baseline Analysis</a></span></li><li><span><a href="#FXD-Analysis" data-toc-modified-id="FXD-Analysis-3.2">FXD Analysis</a></span></li><li><span><a href="#EVD-Analysis" data-toc-modified-id="EVD-Analysis-3.3">EVD Analysis</a></span></li><li><span><a href="#GZD-Analysis" data-toc-modified-id="GZD-Analysis-3.4">GZD Analysis</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-4">Analysis</a></span><ul class="toc-item"><li><span><a href="#Additional-data" data-toc-modified-id="Additional-data-4.1">Additional data</a></span><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-4.1.1">Overview</a></span></li><li><span><a href="#Top-10" data-toc-modified-id="Top-10-4.1.2">Top 10</a></span></li><li><span><a href="#Bottom-10" data-toc-modified-id="Bottom-10-4.1.3">Bottom 10</a></span></li></ul></li></ul></li></ul></div>

<h1>Eye Gaze Data Processing<span class="tocSkip"></span></h1>

Following metrics from: https://github.com/TheD2Lab/Eye.Tracking.Data.Analysis.For.Tobii.2150/tree/master/src/analysis

- Isaac Cortes
- Reynaldo Suarez
- Chris
- Janki

In [1]:
import pandas as pd
import numpy as np
import random 
import math

# Preparation

## Load files

In [2]:
def load_files(participant = 1):
    
    route = f'datasets/p{participant}/p{participant}'
    
    # Load one of the files to create functions
    baseline_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    baseline = pd.read_csv(f'{route}GZD.txt', sep='\t', names = baseline_cols)
    
    # Load FXD graph and tree
    fxd_cols = ['number', 'time', 'duration', 'screen_x', 'screen_y']

    fxd_graph = pd.read_csv(f'{route}.graphFXD.txt', sep='\t', names = fxd_cols)
    fxd_tree = pd.read_csv(f'{route}.treeFXD.txt', sep='\t', names = fxd_cols)

    # Load EVD graph and tree
    evd_cols = ['time', 'event', 'event_key', 'data1', 'data2', 'description']

    evd_graph = pd.read_csv(f'{route}.graphEVD.txt', sep='\t', names = evd_cols)
    evd_tree = pd.read_csv(f'{route}.treeEVD.txt', sep='\t', names = evd_cols)
    
    # Load GZD graph and tree
    gzd_cols = ['number', 'time', 'l_screen_x', 'l_screen_y', 'l_cam_x', 'l_cam_y', 'l_distance', 
                'l_pupil', 'l_code', 'r_screen_x', 'r_screen_y', 'r_cam_x', 'r_cam_y', 'r_distance', 
                'r_pupil', 'r_code']

    gzd_graph = pd.read_csv(f'{route}.graphGZD.txt', sep='\t', names = gzd_cols)
    gzd_tree = pd.read_csv(f'{route}.treeGZD.txt', sep='\t', names = gzd_cols)

    return baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree


In [3]:
# Not all numbers 1-36 are participants in folder, use random choose instead with the folder contents
participant = random.randint(1,36)
print(f'Participant: {participant}')

baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree = load_files(participant)

Participant: 28


# Functions

## Helper functions

$$d = \sqrt{(x_2 - x_1)^2 + (y_2-y_1)^2}$$

In [4]:
def distance(row):
    x1, y1 = row['x'], row['y']
    x2, y2 =  row['next_x'], row['next_y']
    dist = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2))
    row['dist'] = dist
    
    return row

$$dur = T2-(T1+D1)$$

In [5]:
def duration(row):
    
    t1, d1 = row['time'], row['duration']
    t2 = row['next_time']
    duration = t2 - (t1+d1)
    row['duration_between_fixations'] = duration
    
    return row 

## Metric functions

### All metrics

In [6]:
def get_metrics_df(row):
    
    participant = ''.join(filter(str.isdigit, row['ID']))
    print(participant)
    
    visualization = row['Visualization']
    
    # Load data for participant
    baseline, fxd_graph, fxd_tree, evd_graph, evd_tree, gzd_graph, gzd_tree = load_files(participant)
    
    # Baseline gaze data
    base_avg_size_left, base_avg_size_right, base_avg_size_both = get_gaze_metrics(baseline)
    # Graph data
    if visualization == 1:
        # Fixations data
        total_fixations, sum_fixation_duration, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(fxd_graph)
        # Saccade length data
        total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(fxd_graph)
        # Saccade duration data
        sum_saccade_duration, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(fxd_graph)
        # Event data
        total_left_clicks, mean_time_between_clicks, std_time_between_clicks = get_event_metrics(evd_graph)
        # Gaze data
        avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(gzd_graph)
    # Tree data
    else:
        # Fixations data
        total_fixations, sum_fixation_duration, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(fxd_tree)
        # Saccade length data
        total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(fxd_tree)
        # Saccade duration data
        sum_saccade_duration, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(fxd_tree)
        # Event data
        total_left_clicks, mean_time_between_clicks, std_time_between_clicks = get_event_metrics(evd_tree)
        # Gaze data
        avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(gzd_tree)
    
    cols = ['base_avg_size_left', 'base_avg_size_right', 
            'base_avg_size_both', 'total_fixations', 'sum_fixation_duration', 
            'mean_fixation_duration', 'std_fixation_duration', 'total_saccades', 
            'sum_saccade_length', 'mean_saccade_length', 'std_saccade_length', 
            'sum_saccade_duration', 'mean_saccade_duration', 'std_saccade_duration', 
            'total_left_clicks', 'mean_time_between_clicks', 'std_time_between_clicks', 
            'avg_size_left', 'avg_size_right', 'avg_size_both']
    
    data = [base_avg_size_left, base_avg_size_right, 
            base_avg_size_both, total_fixations, sum_fixation_duration, 
            mean_fixation_duration, std_fixation_duration, total_saccades, 
            sum_saccade_length, mean_saccade_length, std_saccade_length, sum_saccade_duration, 
            mean_saccade_duration, std_saccade_duration, total_left_clicks, 
            mean_time_between_clicks, std_time_between_clicks, 
            avg_size_left, avg_size_right, avg_size_both]
    
    for i in range(len(cols)):
        row[cols[i]] = data[i]
    
    return row

### Gaze data

In [7]:
def get_gaze_metrics(df):
    
    # A code with 0 indicates the eye tracker was confdident with this data
    # Filtering only records where both pupil sizes are valid
    df = df[(df['l_code'] == 0) & (df['r_code'] == 0)]
    avg_size_left = df["l_pupil"].mean()
    avg_size_right = df["r_pupil"].mean()
    avg_size_both = pd.concat([df["r_pupil"],df["l_pupil"]]).mean()
    
    return avg_size_left, avg_size_right, avg_size_both

In [8]:
def print_gaze_metrics(df):
    avg_size_left, avg_size_right, avg_size_both = get_gaze_metrics(df)

    print(f'Avg. pupil size of the left eye: {avg_size_left:.4f}')
    print(f'Avg. pupil size of the right eye: {avg_size_right:.4f}')
    print(f'Avg. pupil size of both eyes: {avg_size_both:.4f}')

### Fixation data

In [9]:
def get_fixation_metrics(df):
    
    total_fixations = len(df)
    sum_fixation_duration_sec = df["duration"].sum() / 1000
    mean_fixation_duration = df["duration"].mean()
    std_fixation_duration = df["duration"].std()
    
    return total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration
    
def get_saccade_length_metrics(df):
    
    # Get distances of all points
    coords = df[['screen_x','screen_y']].copy()
    coords = coords.rename({'screen_x':'x','screen_y':'y'}, axis='columns')
    coords['next_x'] = coords['x'].shift(-1)
    coords['next_y'] = coords['y'].shift(-1)
    coords = coords.apply(distance, axis=1)
    coords.dropna(inplace=True)
    
    total_saccades = len(coords)
    sum_saccade_length = coords['dist'].sum()
    mean_saccade_length = coords['dist'].mean()
    std_saccade_length = coords['dist'].std()
        
    return total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length

def get_saccade_durations_metrics(df):
    
    # Get duration of all saccades
    saccadeDetails = df[['time','duration']].copy()
    saccadeDetails['next_time'] = saccadeDetails['time'].shift(-1)
    saccadeDetails['next_duration'] = saccadeDetails['duration'].shift(-1)
    saccadeDetails = saccadeDetails.apply(duration, axis=1)
    saccadeDetails.dropna(inplace=True)
    
    sum_saccade_duration_sec = saccadeDetails['duration_between_fixations'].sum() / 1000
    mean_saccade_duration = saccadeDetails['duration_between_fixations'].mean()
    std_saccade_duration = saccadeDetails['duration_between_fixations'].std()
    
    return sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration

In [10]:
def print_fixation_metrics(df):
    # Fixations
    total_fixations, sum_fixation_duration_sec, mean_fixation_duration, std_fixation_duration = get_fixation_metrics(df)
    print(f'{"="*10} {"Fixations":^20} {"="*10}')
    print(f'Total number of fixations: {total_fixations}')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec}s')
    print(f'Sum of all fixation durations: {sum_fixation_duration_sec / 60 :.2f}min')
    print(f'Mean fixation duration: {mean_fixation_duration:.2f}ms')
    print(f'StDev fixation duration: {std_fixation_duration:.2f}ms')

    # Saccade 
    total_saccades, sum_saccade_length, mean_saccade_length, std_saccade_length = get_saccade_length_metrics(df)
    print(f'\n{"="*10} {"Saccade lengths":^20} {"="*10}')
    print(f'Total number of saccades: {total_saccades}')
    print(f'Sum of all saccade lengths: {sum_saccade_length:.2f}px')
    print(f'Mean saccade length: {mean_saccade_length:.2f}px')
    print(f'StDev saccade length: {std_saccade_length:.2f}px')

    # Durations
    sum_saccade_duration_sec, mean_saccade_duration, std_saccade_duration = get_saccade_durations_metrics(df)
    print(f'\n{"="*10} {"Saccade durations":^20} {"="*10}')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec:.2f}s')
    print(f'Sum of all saccade durations: {sum_saccade_duration_sec / 60 :.2f}min')
    print(f'Mean saccade duration: {mean_saccade_duration:.2f}ms')
    print(f'StDev saccade duration: {std_saccade_duration:.2f}ms')

### Event data

In [11]:
def get_event_metrics(df):
    
    lclicks = evd_graph[evd_graph['event'] == 'LMouseButton']
    lclicks = lclicks[['time','data1','data2']]
    lclicks['next_time'] = lclicks['time'].shift(-1)
    lclicks['time_between'] = lclicks['next_time'] - lclicks['time']
    lclicks.dropna(inplace=True)

    total_L_clicks = len(lclicks)
    avg_time_between_clicks_sec = lclicks['time_between'].mean() / 1000
    std_time_between_clicks_sec = lclicks['time_between'].std() / 1000
    
    return total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec

In [12]:
def print_event_metrics(df):
    
    total_L_clicks, avg_time_between_clicks_sec, std_time_between_clicks_sec = get_event_metrics(df)
    
    print(f'Total number of L mouse clicks: {total_L_clicks}')
    print(f'Avg. amount of time between clicks: {avg_time_between_clicks_sec:.2f}s')
    print(f'StDev amount of time between clicks: {std_time_between_clicks_sec:.2f}s')

# Analysis

## Baseline Analysis

- average pupil size of left eye; 
- average pupil size of right eye; 
- average pupil size of both eyes.

In [13]:
baseline.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,8,1,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
1,28,2,-1280,-1024,0.587,0.276,-1.000,-1.000,4,-1280,-1024,0.348,0.297,-1.000,-1.000,4
2,48,3,-1280,-1024,0.348,0.297,-1.000,-1.000,4,773,370,0.348,0.297,668.391,4.693,0
3,68,4,740,323,0.587,0.276,675.288,4.891,0,783,356,0.348,0.297,668.391,4.765,0
4,88,5,759,323,0.587,0.276,675.288,4.913,0,760,366,0.348,0.298,668.391,4.693,0


In [14]:
%%time
print_gaze_metrics(baseline)

Avg. pupil size of the left eye: 4.3297
Avg. pupil size of the right eye: 3.9673
Avg. pupil size of both eyes: 4.1485
Wall time: 13 ms


## FXD Analysis

**Fixations**
- total number of fixations
- sum of all fixation duration
- mean duration
- StDev of durations

**Saccade lengths**
- total number of saccades
- sum of all saccade length
- mean saccade length
- StDev of saccade lengths

**Saccade durations**
- sum of all saccade durations
- mean saccade duration
- StDev of saccade durations

**Research**
- scanpath duration
- fixation to saccade ratio

In [15]:
fxd_graph.head()

,number,time,duration,screen_x,screen_y
0,1,99,120,674,352
1,2,238,140,685,365
2,3,398,319,675,374
3,4,1155,100,501,156
4,5,1275,100,493,159


In [16]:
%%time
print_fixation_metrics(fxd_graph)

==========      Fixations       ==========
Total number of fixations: 1023
Sum of all fixation durations: 207.278s
Sum of all fixation durations: 3.45min
Mean fixation duration: 202.62ms
StDev fixation duration: 124.58ms

==========   Saccade lengths    ==========
Total number of saccades: 1022
Sum of all saccade lengths: 174510.74px
Mean saccade length: 170.75px
StDev saccade length: 178.79px

==========  Saccade durations   ==========
Sum of all saccade durations: 378.13s
Sum of all saccade durations: 6.30min
Mean saccade duration: 369.99ms
StDev saccade duration: 1300.26ms
Wall time: 1.57 s


In [17]:
fxd_tree.head()

,number,time,duration,screen_x,screen_y
0,1,1040,100,541,126
1,2,1319,100,128,183
2,3,1459,239,64,185
3,4,1818,100,280,164
4,5,1937,140,288,183


In [18]:
%%time
print_fixation_metrics(fxd_tree)

==========      Fixations       ==========
Total number of fixations: 1849
Sum of all fixation durations: 371.402s
Sum of all fixation durations: 6.19min
Mean fixation duration: 200.87ms
StDev fixation duration: 119.54ms

==========   Saccade lengths    ==========
Total number of saccades: 1848
Sum of all saccade lengths: 223016.47px
Mean saccade length: 120.68px
StDev saccade length: 175.90px

==========  Saccade durations   ==========
Sum of all saccade durations: 397.33s
Sum of all saccade durations: 6.62min
Mean saccade duration: 215.00ms
StDev saccade duration: 299.34ms
Wall time: 2.4 s


## EVD Analysis

- total number of L mouse clicks. 
- avg time between clicks.
- std time between clicks.

In [19]:
evd_graph.head()

,time,event,event_key,data1,data2,description
0,7013,LMouseButton,1,515,223,NaN
1,8977,LMouseButton,1,611,214,NaN
2,10850,LMouseButton,1,551,243,NaN
3,24924,LMouseButton,1,386,971,NaN
4,25821,LMouseButton,1,396,980,NaN


In [20]:
%%time
print_event_metrics(evd_graph)

Total number of L mouse clicks: 152
Avg. amount of time between clicks: 3.69s
StDev amount of time between clicks: 6.10s
Wall time: 5 ms


In [21]:
evd_tree.head()

,time,event,event_key,data1,data2,description
0,3352,LMouseButton,1,516,217,NaN
1,6423,LMouseButton,1,661,218,NaN
2,7280,LMouseButton,1,645,233,NaN
3,10540,LMouseButton,1,38,578,NaN
4,11736,LMouseButton,1,55,587,NaN


In [22]:
%%time
print_event_metrics(evd_tree)

Total number of L mouse clicks: 152
Avg. amount of time between clicks: 3.69s
StDev amount of time between clicks: 6.10s
Wall time: 8.56 ms


## GZD Analysis

- average pupil size of left eye;
- average pupil size of right eye;
- average pupil size of both eyes.

In [23]:
gzd_graph.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,19,1,-1280,-1024,0.628,0.275,-1.000,-1.000,4,-1280,-1024,0.391,0.284,-1.000,-1.000,4
1,39,2,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
2,59,3,650,345,0.628,0.276,682.845,4.684,0,657,408,0.391,0.284,677.576,4.494,0
3,79,4,672,319,0.628,0.275,682.845,4.911,0,687,365,0.391,0.284,677.576,4.586,0
4,99,5,666,354,0.628,0.275,682.845,4.778,0,649,393,0.391,0.284,677.576,4.510,0


In [24]:
%%time
print_gaze_metrics(gzd_graph)

Avg. pupil size of the left eye: 3.9297
Avg. pupil size of the right eye: 3.7185
Avg. pupil size of both eyes: 3.8241
Wall time: 7.58 ms


In [25]:
gzd_tree.head()

,number,time,l_screen_x,l_screen_y,l_cam_x,l_cam_y,l_distance,l_pupil,l_code,r_screen_x,r_screen_y,r_cam_x,r_cam_y,r_distance,r_pupil,r_code
0,3,1,-1280,-1024,-1.000,-1.000,-1.00,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
1,23,2,-1280,-1024,0.595,0.374,-1.00,-1.000,4,-1280,-1024,0.379,0.384,-1.000,-1.000,4
2,43,3,-1280,-1024,-1.000,-1.000,-1.00,-1.000,4,-1280,-1024,-1.000,-1.000,-1.000,-1.000,4
3,63,4,756,258,0.597,0.374,746.09,4.980,0,700,336,0.380,0.384,747.556,4.279,0
4,83,5,663,248,0.597,0.374,746.09,4.823,0,704,277,0.381,0.384,747.556,4.504,0


In [26]:
%%time
print_gaze_metrics(gzd_tree)

Avg. pupil size of the left eye: 4.2970
Avg. pupil size of the right eye: 4.1502
Avg. pupil size of both eyes: 4.2236
Wall time: 8.04 ms


# Analysis

## Additional data

In [27]:
add_data = pd.read_csv('datasets/additional_participant_data.csv')

In [28]:
add_data.head()

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p1,1,1,0.608696,26
1,p1,2,2,0.521739,38
2,p3,1,1,0.478261,10
3,p3,2,2,0.391304,50
4,p5,1,1,0.521739,22


### Overview

In [29]:
add_data.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.494565     17.625000
           2                  0.501449     22.000000
2          1                  0.513043     24.666667
           2                  0.407609     35.187500

### Top 10

In [30]:
top10 = add_data.copy()
top10.sort_values(by=['Task_Success','Time_On_Task'], ascending=[False,True], inplace=True)
top10.reset_index(drop=True, inplace=True)
top10 = top10.head(10)
top10

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p6,1,2,0.869565,26
1,p18,1,2,0.739130,21
2,p24,2,1,0.739130,25
3,p6,2,1,0.739130,43
4,p12,2,1,0.695652,11
5,p35,1,1,0.608696,9
6,p10,1,2,0.608696,17
7,p17,2,2,0.608696,22
8,p1,1,1,0.608696,26
9,p34,1,2,0.608696,38


In [31]:
top10.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.608696     17.500000
           2                  0.706522     25.500000
2          1                  0.724638     26.333333
           2                  0.608696     22.000000

In [32]:
%%time
top10 = top10.apply(get_metrics_df, axis=1)

6
18
24
6
12
35
10
17
1
34
Wall time: 51.9 s


In [33]:
top10.to_csv('datasets/top10.csv', sep=',')

### Bottom 10

In [34]:
bottom10 = add_data.copy()
bottom10.sort_values(by=['Task_Success','Time_On_Task'], ascending=[True,False], inplace=True)
bottom10.reset_index(drop=True, inplace=True)
bottom10 = bottom10.head(10)
bottom10

,ID,Ontologies,Visualization,Task_Success,Time_On_Task
0,p27,2,2,0.086957,50
1,p7,2,2,0.086957,20
2,p14,2,1,0.217391,28
3,p27,1,1,0.260870,13
4,p32,1,2,0.304348,21
5,p12,1,2,0.304348,17
6,p25,2,2,0.347826,39
7,p11,2,2,0.347826,33
8,p33,2,2,0.347826,27
9,p2,1,2,0.347826,19


In [35]:
bottom10.groupby(by=['Ontologies','Visualization']).mean()

Task_Success  Time_On_Task
Ontologies Visualization                            
1          1                  0.260870          13.0
           2                  0.318841          19.0
2          1                  0.217391          28.0
           2                  0.243478          33.8

In [36]:
%%time
bottom10 = bottom10.apply(get_metrics_df, axis=1)

27
7
14
27
32
12
25
11
33
2
Wall time: 32.7 s


In [37]:
bottom10.to_csv('datasets/bottom10.csv', sep=',')